In [1]:
# 原地修改sparse_tensor的indices
import torch

# 创建一个稀疏张量
values = torch.tensor([1, 2, 3])
indices = torch.tensor([[0, 1, 2],
                        [2, 0, 1]], dtype=torch.long)
shape = torch.Size([3, 3])
sparse_tensor = torch.sparse_coo_tensor(indices, values, shape)
sparse_tensor = sparse_tensor.coalesce()
# 输出原始indices
print("原始indices:")
print(sparse_tensor.indices())

# 直接修改indices的值
sparse_tensor.indices().copy_(sparse_tensor.indices().flip(0))

# 输出修改后的indices
print("修改后的indices:")
print(sparse_tensor.indices())

原始indices:
tensor([[0, 1, 2],
        [2, 0, 1]])
修改后的indices:
tensor([[2, 0, 1],
        [0, 1, 2]])


/home/xyh/miniconda3/envs/cv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# 性能测试：二值化稀疏矩阵边概率
import torch
import time
row, col = 3000, 3000
matrix = torch.rand(row, col)

# 将80%的元素设置为0
mask = torch.rand(row, col) > 0.8
matrix = torch.where(mask, torch.zeros(row, col), matrix)

print(matrix)
sparse = matrix.to_sparse()
print(sparse)

def fun0(matrix):
    threshold = 0.5
    mask = matrix >= threshold
    matrix = mask.float()

    return matrix

def fun1(matrix):
    threshold = 0.5
    adj_new = torch.where(matrix < threshold, torch.zeros_like(matrix), torch.ones_like(matrix))
    return adj_new

def fun2(matrix):
    adj_new = matrix.clone()
    threshold = 0.5
    adj_new[adj_new < threshold] = 0
    adj_new[adj_new >= threshold] = 1
    return adj_new

def fun3(sparse):
    threshold = 0.5
    mask = sparse.values() >= threshold

    sparse.values().copy_(mask)
    return sparse


t = time.time()
for _ in range(100):
    fun0(matrix)
print(time.time() - t)

t = time.time()
for _ in range(100):
    fun1(matrix)
print(time.time() - t)


t = time.time()
for _ in range(100):
    fun2(matrix)
print(time.time() - t)


t = time.time()
for _ in range(100):
    fun3(sparse)
print(time.time() - t)


tensor([[0.3868, 0.0000, 0.9923,  ..., 0.3965, 0.7199, 0.8009],
        [0.0000, 0.2391, 0.0000,  ..., 0.9965, 0.0000, 0.0000],
        [0.8075, 0.1993, 0.0000,  ..., 0.9350, 0.1253, 0.9919],
        ...,
        [0.5672, 0.0000, 0.7320,  ..., 0.3919, 0.5945, 0.0294],
        [0.2481, 0.3313, 0.1918,  ..., 0.0000, 0.0000, 0.2333],
        [0.0114, 0.2290, 0.3581,  ..., 0.0000, 0.5057, 0.0479]])
tensor(indices=tensor([[   0,    0,    0,  ..., 2999, 2999, 2999],
                       [   0,    2,    3,  ..., 2996, 2998, 2999]]),
       values=tensor([0.3868, 0.9923, 0.2019,  ..., 0.5462, 0.5057, 0.0479]),
       size=(3000, 3000), nnz=7199303, layout=torch.sparse_coo)
0.8089625835418701
2.0562031269073486
13.699702262878418
0.5167815685272217


In [14]:
# 根据行列索引取出二维矩阵中对应的元素
import torch

index_matrix = torch.tensor([[0, 1], [1, 3]])
probability_matrix = torch.tensor([[1, 2, 3, 4], [2, 3, 4, 5]])

# 从概率矩阵中取出指定的元素
result = probability_matrix[index_matrix[:, 0], index_matrix[:, 1]]
result = result.view(-1, 1)  # 将结果转换为列向量

print(result)

tensor([[2],
        [5]])


In [17]:
# 测试: 根据标签生成　训练集，验证集和测试集索引
from utils import split_dataset
import numpy as np

labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
labels = np.array(labels)
train_indices, val_indices, test_indices = split_dataset(labels, 0.6, 0.2, 0.2)
print("Training indices and values:", train_indices, labels[train_indices])
print("Validation indices and values:", val_indices, labels[val_indices])
print("Test indices and values:", test_indices, labels[test_indices])

Train set - positive samples: 6
Train set - negative samples: 6

Validation set - positive samples: 2
Validation set - negative samples: 2

Test set - pos samples: 2
Test set - neg samples: 2

Training indices and values: [18 10 15  5 13  9  0  8  2  6  1 12] [0 1 0 0 1 0 1 0 1 0 1 1]
Validation indices and values: [ 4 17 19 14] [1 0 0 1]
Test indices and values: [11  3  7 16] [1 1 0 0]


In [21]:
# 测试: 稀疏矩阵乘法
import torch

# 创建稀疏矩阵A
A = torch.sparse_coo_tensor(
    torch.tensor([[0, 1, 2], [1, 0, 2]], dtype=float),
    torch.tensor([1, 2, 3], dtype=float),
    size=(3, 3)
)

# 创建稀疏矩阵B
B = torch.sparse_coo_tensor(
    torch.tensor([[0, 1, 1], [2, 2, 0]], dtype=float),
    torch.tensor([4, 5, 6], dtype=float),
    size=(3, 3)
)

# 对稀疏矩阵进行乘法运算
C = torch.sparse.mm(A, B)

print(C)

tensor(indices=tensor([[0, 0, 1],
                       [2, 0, 2]]),
       values=tensor([5., 6., 8.]),
       size=(3, 3), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)


In [22]:
# 测试　评估函数
from utils import print_metrics

class_pred = torch.tensor([1, 2, 3, 4, 5, 6, 7])
class_true = torch.tensor([1, 3, 4, 5, 5, 3, 7])
class_real = torch.tensor([0, 3, 3, 5, 5, 3, 6])

# y_true = [1, 0, 1, 0, 0, 0, 1]
# y_pred = [1, 1, 0, 1, 0, 1, 1]

print_metrics(class_pred, class_true, class_real)

(0.42857142857142855, 0.4, 0.6666666666666666, 0.5, tensor(0.6667))

In [23]:
# 由方法和方法的调用矩阵，方法和类的归属矩阵，得到方法和类的调用矩阵
import torch

# 定义方法和方法之间的调用关系矩阵A
A = torch.tensor([[0, 1, 0, 0, 0],
                  [0, 0, 1, 0, 0],
                  [1, 0, 0, 1, 0],
                  [0, 1, 0, 0, 1],
                  [0, 0, 0, 0, 0]])

# 定义方法和类之间的归属关系矩阵B
B = torch.tensor([[1, 0, 0],
                  [0, 1, 0],
                  [1, 0, 0],
                  [0, 0, 1],
                  [0, 0, 1]])

# 计算方法和类之间的邻接矩阵C
C = torch.matmul(A, B)

print(C)

tensor([[0, 1, 0],
        [1, 0, 0],
        [1, 0, 1],
        [0, 1, 1],
        [0, 0, 0]])
